# Aplicações práticas

# 2.1 Exercício 2: Rua e HIV

## A. Enunciado 

Um assistente hospitalar pretende avaliar a associação entre viver ou trabalhar na rua e ter HIV. O assistente selecionou aleatoriamente 454 adolescentes dos registros, dos quais 319 trabalhavam na rua e 135 viviam na rua. Acompanhou-os ao longo de 3 anos. Dos que trabalhavam na rua, 311 foram diagnosticados com HIV e 8 não tinham o vírus; dos que viviam na rua, 123 tinham HIV e 12 não tinham. As variáveis são ambas nominais, pois são definidas apenas pelo nome. As categorias das variáveis são: para a variável Rua (1. Trabalha na rua, 2. Vive na rua), e para a variável HIV (1. Tem HIV, 2. Não tem HIV). O estudo é prospectivo de coorte e os dados observados constam da linha "Count" da Tabela 1, bem como do arquivo "cap02_exercicio 2_Rua e HIV.sav" e organizados em um dicionário em Python.

# Resolução em Python

## Carregando Bibliotecas

In [1]:
import pandas as pd  # Fornece DataFrames e Series para manipulação de dados em Python, facilitando operações como leitura, escrita, e manipulação de estruturas de dados tabulares.
import numpy as np  # Suporta operações com arrays e matrizes, oferecendo uma vasta gama de funções matemáticas para operações com essas estruturas de dados.
from statsmodels.stats.contingency_tables import Table  # Analisa tabelas de contingência para estudo de variáveis categóricas, útil em testes de hipóteses e análises de associação entre variáveis.
from scipy.stats import hypergeom  # Aplica a distribuição hipergeométrica para análises estatísticas, útil em testes de sobreposição e em situações onde se deseja calcular probabilidades sem reposição.

## Funções Personalizadas

In [2]:
import sys

# Adicionar o caminho do diretório ao sys.path
sys.path.append(r"C:\Users\ricar\OneDrive\Área de Trabalho\Livro\Cap_02\socialdataanalysis")

from association import gerar_tabela_contingencia
from association import calcular_odds_ratio_razao_risco_discrepancia
from association import analisar_independencia_variaveis_tabela_contingencia
from association import calcular_distribuicao_probabilidades_e_decisao_hipotese
from association import complementar_tabela_contingencia_com_analise_estatistica 

## Dados Fornecidos

In [3]:
# Dicionário com dados categorizados representando grupos e suas frequências
dados = {
    'Rua': [1.0, 1.0, 2.0, 2.0],
    'HIV': [1.0, 2.0, 1.0, 2.0],
    'Freq': [311.0, 8.0, 123.0, 12.0]
}

# Criando o DataFrame a partir do dicionário
dados = pd.DataFrame(dados)

# Convertendo as colunas para inteiros, se necessário
dados = dados.astype(int)

# Visualizando o dataframe
display(dados)

,Rua,HIV,Freq
0,1,1,311
1,1,2,8
2,2,1,123
3,2,2,12


## Tabela 1: Tabela de Contingência

In [4]:
# Obter automaticamente os nomes dos grupos e da coluna de frequência
grupos = dados.columns[:-1].tolist()  # Todos exceto a última coluna
categorias = {
    grupos[0]: ['1. Trabalha na Rua', '2. Vive na Rua',],
    grupos[1]: ['1. Tem HIV', '2. Não tem HIV'],
}

tabela_contingencia = gerar_tabela_contingencia(dados, grupos, categorias)

display(tabela_contingencia)

HIV,1. Tem HIV,2. Não tem HIV
Rua,,
1. Trabalha na Rua,311,8
2. Vive na Rua,123,12


## Tabela de Contingência com Cálculos

In [5]:
# Função própria complementar_tabela_contingencia_com_analise_estatistica
combined_df = complementar_tabela_contingencia_com_analise_estatistica(tabela_contingencia)

# Visualizando resultado
display(combined_df)

HIV                                      1. Tem HIV 2. Não tem HIV  Total
Rua                                                                      
1. Trabalha na Rua Count                      311.0            8.0  319.0
                   Expected Count             304.9           14.1  319.0
                   % within Rua                97.5            2.5  100.0
                   % within HIV                71.7           40.0   70.3
                   % of Total                  68.5            1.8   70.3
                   Standardized Residual        0.3           -1.6       
                   Adjusted Residual            3.0           -3.0       
2. Vive na Rua     Count                      123.0           12.0  135.0
                   Expected Count             129.1            5.9  135.0
                   % within Rua                91.1            8.9  100.0
                   % within HIV                28.3           60.0   29.7
                   % of Total                  27.1            2.6   29.7
                   Standardized Residual       -0.5            2.5       
                   Adjusted Residual           -3.0            3.0       
Total              Count                      434.0           20.0  454.0
                   Expected Count             434.0           20.0  454.0
                   % within Rua                95.6            4.4  100.0
                   % within HIV               100.0          100.0  100.0
                   % of Total                  95.6            4.4  100.0

<font color="blue">    
    
### 2.2.1. Probabilidades
</font>

<font color="blue">

#### a) Probabilidade de ter HIV sabendo que vive na rua. 

</font>

In [6]:
## PROBABILIDADES CONDICIONADAS ##

# Calculando P(grupo0|grupo1)
prob_cond_grupo0_dado_grupo1 = tabela_contingencia.div(tabela_contingencia.sum(axis=0), axis=1)

# Calculando P(grupo1|grupo0)
prob_cond_grupo1_dado_grupo0 = tabela_contingencia.div(tabela_contingencia.sum(axis=1), axis=0)

# Exibindo as probabilidades condicionais com arredondamento para 3 casas decimais
print(f"Probabilidade Condicionada P({grupos[1]}|{grupos[0]}):")
display(prob_cond_grupo1_dado_grupo0.round(3))

# Impressão de configuração específica
evento = categorias[grupos[1]][0]
condicao = categorias[grupos[0]][1]

print(f"RESPOSTA: \n P({evento}|{condicao}) = {prob_cond_grupo1_dado_grupo0.loc[condicao, evento]:.3f}")

Probabilidade Condicionada P(HIV|Rua):


HIV,1. Tem HIV,2. Não tem HIV
Rua,,
1. Trabalha na Rua,0.975,0.025
2. Vive na Rua,0.911,0.089


RESPOSTA: 
 P(1. Tem HIV|2. Vive na Rua) = 0.911


<font color="blue">

#### b) Probabilidade de ter HIV sabendo que trabalha na rua.
    
</font>

In [7]:
## PROBABILIDADES CONDICIONADAS ##

# Impressão de configuração específica
evento = categorias[grupos[1]][0]
condicao = categorias[grupos[0]][0]

print(f"RESPOSTA: \n P({evento}|{condicao}) = {prob_cond_grupo1_dado_grupo0.loc[condicao, evento]:.3f}")

RESPOSTA: 
 P(1. Tem HIV|1. Trabalha na Rua) = 0.975


<font color="blue">

#### c) Probabilidade de trabalhar na rua.
    
</font>

In [8]:
## PROBABILIDADES MARGINAIS ##

# Calculando as Probabilidades Marginais para o Grupo 1
prob_marg_grupo0 = tabela_contingencia.sum(axis=1) / tabela_contingencia.values.sum()
# Convertendo em um DataFrame
prob_marg_grupo0 = prob_marg_grupo0.to_frame(name='')

# Calculando as Probabilidades Marginais para o Grupo 2
prob_marg_grupo1 = tabela_contingencia.sum(axis=0) / tabela_contingencia.values.sum()
# Convertendo em um DataFrame
prob_marg_grupo1 = prob_marg_grupo1.to_frame(name='').T 

# Exibindo os DataFrames
print(f"Probabilidades Marginais de {grupos[0]}")
display(prob_marg_grupo0.round(3))

# Impressão de configuração específica
evento = categorias[grupos[0]][0]
print(f"RESPOSTA: \n P({evento}) = {prob_marg_grupo0.loc[evento, '']:.3f}")

Probabilidades Marginais de Rua


,
Rua,
1. Trabalha na Rua,0.703
2. Vive na Rua,0.297


RESPOSTA: 
 P(1. Trabalha na Rua) = 0.703


<font color="blue">    
    
### 2.2.2. Frequência esperada
</font>

<font color="blue">
    
#### d) A frequência que se espera obter nos que têm HIV e trabalham na rua.

</font>

In [9]:
## FREQUÊNCIAS ESPERADAS ##

# Encapsulando a tabela para análises no statsmodels
tabela_analise = Table(tabela_contingencia)

# Calculando as frequências esperadas (fe)
frequencias_esperadas = tabela_analise.fittedvalues

# Exibindo as fe
display(frequencias_esperadas.round(1))

# Impressão de configuração específica
evento_grupo0 = categorias[grupos[0]][0]
evento_grupo1 = categorias[grupos[1]][0]
print(f"RESPOSTA: \n Fe({evento_grupo0}, {evento_grupo1}) = {frequencias_esperadas.loc[evento_grupo0, evento_grupo1]:.1f}")

HIV,1. Tem HIV,2. Não tem HIV
Rua,,
1. Trabalha na Rua,304.9,14.1
2. Vive na Rua,129.1,5.9


RESPOSTA: 
 Fe(1. Trabalha na Rua, 1. Tem HIV) = 304.9


<font color="blue">
    
### 2.2.3. Risco Relativo
</font>

<font color="blue">

#### e) A intensidade da associação entre as duas variáveis, recorrendo ao risco relativo, por o estudo ser prospetivo de coorte. Calcule a discrepância entre OR e RR.

</font>

In [10]:
calcular_odds_ratio_razao_risco_discrepancia(tabela_contingencia, print_results=True)

,Value,95% CI Lower,95% CI Upper
Odds Ratio for Rua (1. Trabalha na Rua / 2. Vive na Rua),3.793,1.514,9.504
RR (ou RP) for HIV = 1. Tem HIV,1.070,1.012,1.131
RR (ou RP) for HIV = 2. Não tem HIV,0.282,0.118,0.675
N. of Valid Cases,454,,


Discrepância = 2.544 e (θ * p21) = 3.456


<font color="blue">
    
### 2.2.4. Resíduos Ajustados Estandardizados
</font>

<font color="blue">

#### f)	A significância da associação, usando os resíduos ajustados estandardizados.

</font>

In [12]:
# Resíduos de Pearson (não ajustados)
residuos_nao_ajustados = tabela_analise.resid_pearson

# Resíduos ajustados estandardizados
residuos_estandardizados = tabela_analise.standardized_resids

# Visualizando
display(residuos_estandardizados.round(1))

HIV,1. Tem HIV,2. Não tem HIV
Rua,,
1. Trabalha na Rua,3.0,-3.0
2. Vive na Rua,-3.0,3.0


<font color="blue">
    
### 2.2.5. Probabiblidade de todos os acontecimentos possíveis
</font>

<font color="blue">

#### g) A probabilidade dos acontecimentos observados na Tabela 1.

</font>

In [14]:
# Valores observados na amostra
observed_a11 = tabela_contingencia.iloc[0, 0]
observed_a12 = tabela_contingencia.iloc[0, 1]
observed_a21 = tabela_contingencia.iloc[1, 0]
observed_a22 = tabela_contingencia.iloc[1, 1]

# Definição dos totais marginais
M = tabela_contingencia.sum().sum() # total da amostra
n = observed_a11 + observed_a12 # total marginal da linha
N = observed_a11 + observed_a21 # total marginal da coluna

# Frequências esperadas
expected_a11 = frequencias_esperadas.iloc[0, 0]
# O inteiro mais próximo abaixo de expected_a11 é obtido pelo floor (arredondamento para baixo)
expected_a11_nearest_lower_integer = int(expected_a11)
# O inteiro mais próximo acima de expected_a11 é obtido pelo ceil (arredondamento para cima)
expected_a11_nearest_higher_integer = int(expected_a11) + 1

# Calculando a probabilidade do acontecimento observado
p_observed = hypergeom.pmf(observed_a11, M, n, N)

# Calculando as probabilidades dos dois acontecimentos mais prováveis
p_expected_lower = hypergeom.pmf(expected_a11_nearest_lower_integer, M, n, N)
p_expected_higher = hypergeom.pmf(expected_a11_nearest_higher_integer, M, n, N)

# Imprimindo
print(f"\nProbabilidade do acontecimento observado:")
print(f"P(a11={observed_a11}) = {p_observed:.4f}")


Probabilidade do acontecimento observado:
P(a11=311) = 0.0030


<font color="blue">

#### h) Gerar probabilidades dos acontecimentos possíveis, e determinar as regiões de aceitação e de rejeição da H0

</font>

In [15]:
df_formated, acceptance_range, rejection_range = calcular_distribuicao_probabilidades_e_decisao_hipotese(tabela_contingencia, 0.05)

display(df_formated.style.hide(axis='index'))

print(f"Região de Aceitação: {acceptance_range}")
print(f"Região de Rejeição: {rejection_range}")

y_value,p_value,cum_sum
305,0.196,0.196
304,0.184,0.380
306,0.165,0.545
303,0.133,0.678
307,0.112,0.790
302,0.072,0.862
308,0.062,0.924
309,0.028,0.952
301,0.027,0.979
310,0.010,0.989


Região de Aceitação: [302, 309]
Região de Rejeição: [299, 301] U [310, 319]


<font color="blue">    
    
### 2.2.6. Teste de Fisher e hipóteses
</font>

<font color="blue">

#### i) Associação entre as variáveis usando o teste exato de Fisher, por a amostra ser desequilibrada.
#### j) O valor do nível de significância do teste, usando os resultados da syntax
</font>

In [16]:
analisar_independencia_variaveis_tabela_contingencia(tabela_contingencia, 
                                                     mostrar_pearson=True, 
                                                     mostrar_continuity=False, 
                                                     mostrar_likelihood=True, 
                                                     mostrar_fisher=True)

,Value,df,Asymp. Sig. (2-sided),Exact Sig. (2-sided),Exact Sig. (1-sided)
Test,,,,,
Pearson Chi-Square,9.17,1,0.002,,
Likelihood Ratio,8.24,1,0.004,,
Fisher`s Exact Test,3.79,,,0.005,0.004
N. of Valid Cases,454,,,,


a. 0 cells (0.00%) have expected count less than 5. The minimum expected count is 5.95.
